In [92]:
%matplotlib inline
import csv
import networkx as nx
from networkx.algorithms import bipartite
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

In [47]:
DATA_DIR = "./data/"
CASES_2010_2014_CSV = "CRM Cases 2010_2014 Unrestricted.csv"
CASES_2015_2019_CSV = "CRM Cases 2015_2019 Unrestricted.csv"
ECOMETRICS_VERTICAL_CSV = "CRM CBG Ecometrics Vertical.csv"
CASE_TYPES_CSV = "CRM Case Types.csv"
SHAPEFILE = "./Block+Groups+2010+BARI/Census Block Groups"

DEMOGRAPHICS = "./CT_info.csv"

In [117]:
weights = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for filename in [CASES_2015_2019_CSV, CASES_2010_2014_CSV]:
    with open(DATA_DIR + filename, 'r') as f:
        c = csv.DictReader(f, delimiter='\t')
        count = 0
        for row in c:
            count += 1
            open_dt = row.get('OPEN_DT')
            close_dt = row.get('CLOSED_DT')
            reason = row.get('TYPE')
            ct = row.get('CT_ID_10')
            if "NA" in (reason, ct):
                continue
            weights[open_dt[:4]][ct][reason.lower()] += 1
        

In [101]:
ct_info = defaultdict(dict)
with open(DEMOGRAPHICS, 'r') as f:
    c = csv.DictReader(f, delimiter=',')
    for row in c:
        ct_info[row.get('GEOID10')]['TotalPop'] = row['POP100_']
        ct_info[row.get('GEOID10')]['MedHouseIncome'] = row['MdHsInc']
        ct_info[row.get('GEOID10')]['MedHomeVal'] = row['MedHmVl']

with open("output.csv", "w") as f:
    out = csv.DictWriter(f, ["CT", "Type", "Calls", "TotalPop", "MedHouseIncome", "MedHomeVal", "CallsPerPop"])
    out.writeheader()
    for year in weights:
        for ct in weights[year]:
            for reason in weights[year][ct]:
                count = weights[year][ct][reason]
                
                if ct not in ct_info:
                    print(ct)
                    continue
                
                info = ct_info[ct]
                if int(info['TotalPop']) == 0:
                    print("0 Pop")
                    print(ct)
                    continue
                    
                out.writerow({
                    "CT": ct,
                    "Type": reason,
                    "Calls": count,
                    "TotalPop": info["TotalPop"],
                    "MedHouseIncome": info["MedHouseIncome"],
                    "MedHomeVal": info["MedHomeVal"],
                    "CallsPerPop": float(count) / float(info["TotalPop"])
                })
                
    
    

0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981700
0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981600
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
25025981201
0 Pop
2502

In [162]:
def build_graphs_from_weights(weights):
    l = defaultdict(lambda: nx.Graph())
    for year in weights:
        for ct in weights[year]:
            l[year].add_node(ct, bipartite='ct')
            for reason in weights[year][ct]:
                l[year].add_node(reason, bipartite='reason')
                l[year].add_edge(reason, ct, weight=weights[year][ct][reason])
    return l
Gs = build_graphs_from_weights(weights)
    

In [163]:
def export_bipartite(G):
    ct_count = sum([1 for n in G.nodes if G.nodes[n]['bipartite'] == 'ct'])
    ordering = sorted(G.nodes)
    mat = nx.to_numpy_matrix(G, ordering, weight='weight')
    return mat[ct_count:len(ordering), 0:ct_count], ordering[:ct_count], ordering[ct_count:]


In [164]:
adjs = {year: export_bipartite(Gs[year]) for year in Gs}

In [165]:
for year, (adj, _, _) in adjs.items():
    np.savetxt(year + ".txt", adj, fmt="%f") # TODO change maybe - this always integer
    

In [6]:
def get_nodes(G, **kwargs):
    return [n for n in G.nodes if (all(G.nodes[n][key] == kwargs[key] for key in kwargs))]

In [7]:
projected = {}
for year in Gs:
    projected[year] = bipartite.weighted_projected_graph(Gs[year], get_nodes(Gs[year], bipartite='ct'))

In [17]:
for year in projected:
    print(len(nx.community.modularity_max.greedy_modularity_communities(projected[year], 'weight')))
    

554
554
11
555
553
552
549


In [79]:
def get_frequencies(G, t):
    return sorted(G.degree(get_nodes(G, bipartite=t), weight='weight'), key=lambda x: x[1], reverse=True)

#get_frequencies(Gs['2011'], 'reason')

In [80]:

#get_frequencies(Gs['2011'], 'ct')

In [81]:
#print(get_nodes(Gs['2014'], bipartite='reason'))

In [14]:
export_graph(projected['2014'], '2014')

In [11]:
def export_graph(G, file):
    nx.write_weighted_edgelist(G, file, delimiter=";")
        
#export_graph(bipartite.weighted_projected_graph(G, reason_nodes), "test.txt")

In [13]:
def draw_bipartite(G):
    X, Y = bipartite.sets(G)
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) 
    nx.draw(G, pos=pos)
    plt.show()


In [1]:
def setup_map(shapefile, name, scale):
    from mpl_toolkits.basemap import Basemap
    plt.figure(figsize=(12*scale,6*scale))
    m = Basemap(projection='merc', llcrnrlat=42.22, urcrnrlat=42.42,
                   llcrnrlon=-71.2, urcrnrlon=-70.98, resolution='h')
    m.readshapefile(shapefile, name, linewidth=0.7)
    m.drawcoastlines()
    m.drawstates()
    m.drawmapboundary(fill_color='aqua')
    m.fillcontinents(color='coral',lake_color='aqua')
    return m
    

#m = setup_map(SHAPEFILE, "CBGs", 2)
#try:
   # m.plot()
#except Exception as e:
    #pass

